<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Equipo Docente IIC2233 2019-2 y basado en material creado en 2017-2 por Hugo Navarrete e Ignacio Acevedo.</font>
</p>

## *Threads* y PyQt

El uso de *theading* en conjunto a PyQt es completamente compatible. En un cuaderno anterior se mostró un ejemplo inicial del uso de *threading* para actualizar una interfaz de usuario a través de señales. El ejemplo instancia un *thread* que envía señales a la ventana para que actualice el contenido de esta:

In [ ]:
import sys
from threading import Thread
from time import sleep

from PyQt5.QtCore import pyqtSignal
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QHBoxLayout,
                             QVBoxLayout, QPushButton)


class MiThread(Thread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    def __init__(self, senal_actualizar, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.senal_actualizar = senal_actualizar

    def run(self):
        for i in range(10):
            sleep(0.5)
            self.senal_actualizar.emit(str(i))
        self.senal_actualizar.emit('Status: thread terminado')


class MiVentana(QWidget):

    # Creamos una señal para manejar la respuesta del thread
    senal_thread = pyqtSignal(str)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.thread = None
        # Conectamos la señal del thread al método que maneja
        self.senal_thread.connect(self.actualizar_labels)

        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        self.label = QLabel('Status: esperando thread', self)
        self.boton = QPushButton('Ejecutar Thread', self)
        self.boton.clicked.connect(self.ejecutar_threads)

        hbox1 = QHBoxLayout()
        hbox1.addStretch(1)
        hbox1.addWidget(self.label)
        hbox1.addStretch(1)

        hbox2 = QHBoxLayout()
        hbox2.addStretch(1)
        hbox2.addWidget(self.boton)
        hbox2.addStretch(1)

        vbox = QVBoxLayout()
        vbox.addStretch(1)
        vbox.addLayout(hbox1)
        vbox.addStretch(1)
        vbox.addLayout(hbox2)
        vbox.addStretch(1)
        self.setLayout(vbox)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle('Ejemplo threads')
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self):
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        if self.thread is None or not self.thread.is_alive():
            self.thread = MiThread(self.senal_thread)
            self.thread.start()

    def actualizar_labels(self, evento):
        """
        Este método actualiza el label según los datos enviados desde el
        thread através del objeto evento. Para este ejemplo, el método
        recibe el evento, pero podría también no recibir nada.
        """
        self.label.setText(evento)


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec_())

Con esto, lo que sabemos y conocemos del módulo `threading` nos basta para generar comportamiento *multithread* en conjunto con nuestras interfaces. Pero, hay más. Debido a que PyQt se construye sobre la **arquitectura basada en manejo de eventos**, *threads* comunes y corrientes pueden generar problemas de concurrencia con esta arquitectura, por lo que PyQt provee su propia implementación de *threads*: el `QThread`.

## `QThread`

En términos simples, es un `Thread`. Provee prácticamente las mismas funcionlidades a las que estamos acostrumbrados, pero está creado dentro del ambiente de PyQt, por lo que es mucho más compatible. Es parte del módulo `PyQt5.QtCore`. A continuación se muestra una recreación del ejemplo anterior utilizando `QThread` en vez de `Thread`:

In [ ]:
import sys
from time import sleep
from PyQt5.QtCore import pyqtSignal, QThread
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QHBoxLayout,
                             QVBoxLayout, QPushButton)


class MiThread(QThread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    def __init__(self, senal_actualizar, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.senal_actualizar = senal_actualizar

    def run(self):
        for i in range(10):
            sleep(0.5)
            self.senal_actualizar.emit(str(i))
        self.senal_actualizar.emit('Status: thread terminado')


class MiVentana(QWidget):

    # Creamos una señal para manejar la respuesta del thread
    senal_thread = pyqtSignal(str)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.thread = None
        # Conectamos la señal del thread al método que maneja
        self.senal_thread.connect(self.actualizar_labels)

        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        self.label = QLabel('Status: esperando thread', self)
        self.boton = QPushButton('Ejecutar Thread', self)
        self.boton.clicked.connect(self.ejecutar_threads)

        hbox1 = QHBoxLayout()
        hbox1.addStretch(1)
        hbox1.addWidget(self.label)
        hbox1.addStretch(1)

        hbox2 = QHBoxLayout()
        hbox2.addStretch(1)
        hbox2.addWidget(self.boton)
        hbox2.addStretch(1)

        vbox = QVBoxLayout()
        vbox.addStretch(1)
        vbox.addLayout(hbox1)
        vbox.addStretch(1)
        vbox.addLayout(hbox2)
        vbox.addStretch(1)
        self.setLayout(vbox)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle('Ejemplo threads')
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self):
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        if self.thread is None or not self.thread.isRunning():
            self.thread = MiThread(self.senal_thread)
            self.thread.start()

    def actualizar_labels(self, evento):
        """
        Este método actualiza el label según los datos enviados desde el
        thread através del objeto evento. Para este ejemplo, el método
        recibe el evento, pero podría también no recibir nada.
        """
        self.label.setText(evento)


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec_())

Se puede apreciar que se define de la misma forma que un `Thread`: se llama a `__init__` de la clase ancestro de la cual hereda, se define el método `run` que define el comportamiento al ser ejecutado, y este se ejecuta al llamar el método `start`. Una diferencia leve que se puede notar es que el método `is_alive` no existe en `QThread`, pero existe un equivalente llamado `isRunning`. En general se puede encontrar un equivalente para todo método de `Threads` en `QThreads`, esta se puede encontrar buscando documentación de la clase en la web.

Un aspecto notable a considerar es que `QThread` hereda de `QObject`, al igual que la mayoría de los objetos que hemos usado de PyQt. Luego, esto nos permite de fácil manera crear señales para múltiples *threads* dentro de una clase (también es posible usando `Thread`s normales, ¿se te ocurre cómo?):

In [ ]:
import sys
from time import sleep

from PyQt5.QtCore import pyqtSignal, QThread
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QHBoxLayout,
                             QVBoxLayout, QPushButton)


class MiThread(QThread):
    
    # Se define para la clase MiThread,
    # para que cada instancia tenga una propia
    senal_actualizar = pyqtSignal(int, str)

    def __init__(self, i, tiempo, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.indice = i
        self.tiempo = tiempo

    def run(self):
        for i in range(10):
            sleep(self.tiempo)
            self.senal_actualizar.emit(self.indice, str(i))
        self.senal_actualizar.emit(self.indice, 'Status: thread terminado')


class MiVentana(QWidget):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.threads = []
        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        # Definimos un montón de labels que corresponderán a un thread cada uno
        self.labels = {i : QLabel('Status: esperando thread', self) for i in range(1, 6)}
        self.boton = QPushButton('Ejecutar Threads', self)
        self.boton.clicked.connect(self.ejecutar_threads)

        hboxs = []
        for i in range(1, 6):
            hbox = QHBoxLayout()
            hbox.addStretch(1)
            hbox.addWidget(self.labels[i])
            hbox.addStretch(1)
            hboxs.append(hbox)

        hbox = QHBoxLayout()
        hbox.addStretch(1)
        hbox.addWidget(self.boton)
        hbox.addStretch(1)
        hboxs.append(hbox)

        vbox = QVBoxLayout()
        for hbox in hboxs:
            vbox.addStretch(1)
            vbox.addLayout(hbox)
        vbox.addStretch(1)
        self.setLayout(vbox)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle('Ejemplo threads')
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self):
        """
        Este método crea cinco threads cada vez que se presiona el botón en la
        interfaz. Los threads recibirás como argumento el índice del label 
        que les corresponde y el tiempo que toman entre cada iteración.
        """
        if len(self.threads) == 0 or not any([thread.isRunning() for thread in self.threads]):
            self.threads = []
            for i in range(1, 6):
                thread = MiThread(i, i / 10)
                # Se conecta la señal emitida por el thread a un método
                # de la ventana
                thread.senal_actualizar.connect(self.actualizar_labels)
                self.threads.append(thread)
                thread.start()

    def actualizar_labels(self, indice, texto):
        """
        Este método actualiza el label correspondiente según los datos 
        enviados desde un thread através del índice y aplica el texto.
        """
        self.labels[indice].setText(texto)


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec_())

## `QTimer`

Otra herramienta que existe para generar comportamiento de concurrencia dentro de PyQt es la clase `QTimer`. Los objetos de esta clase, a diferencia de los `QThread`s, no es un símil a `Timer` revisado en el apunte anterior. `Timer` después de un tiempo específico, ejecuta una subrutina una única vez. Mientras que `QTimer` ejecuta una subrutina cada cierto tiempo determinado periodicamente, repitiendo la subrutina una y otra vez.

Este tipo de comportamiento es simulable utilizando `QThread`. Al definir código para un *thread* de la forma:

```python
def run(self):
    while True:
        # Lo que quiero que el QThread haga en cada iteración
        time.sleep(tiempo)
```

Y debería obtener el mismo resultado a lo descrito para `QTimer`, y es cierto. La desventaja de realizar esto con `QTimer` es que estos están construidos para efectuar este comportamiento, mientras que los `QThreads`, como los *threads* en general, están contruidos para que acaben eventualmente. Luego, el implementar un *thread* usando el código anterior nos obliga a usar el método `terminate`, que se considera **mala práctica** al forzar un *thread* a terminar, en vez de que este termine por su cuenta.

Por su lado, `QTimer` provee métodos para comenzar (`start`) y detener (`stop`) la ejecución periodica de buena manera. Tras inicializar un `QTimer`, se le asigna mediante `setInterval` el tiempo en milisegundos que durará el periodo entre ejecuciones, y mediante el atributo (y señal) `timeout` se puede conectar a la subrutina que se efectuará una y otra vez: `timer.timeout.connect(subrutina)`.

A continuación se muestra un ejemplo adaptando el anterior pero con el uso de `QTimer` en vez de `QThread`:



In [ ]:
import sys
from time import sleep

from PyQt5.QtCore import pyqtSignal, QTimer, QObject
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QHBoxLayout,
                             QVBoxLayout, QPushButton)


class MiTimers(QObject):
    
    senal_actualizar = pyqtSignal(int, str)

    def __init__(self, i, tiempo):
        super().__init__()
        self.indice = i
        self.tiempo = tiempo
        self.indice_actual = 0
        self.timer = QTimer()
        self.timer.setInterval(tiempo * 1000)
        self.timer.timeout.connect(self.enviar_dato)

    def enviar_dato(self):
        if self.indice_actual < 9:
            self.senal_actualizar.emit(self.indice, str(self.indice_actual))
            self.indice_actual += 1
        else:
            self.senal_actualizar.emit(self.indice, 'Status: timer terminado')
            self.timer.stop()

    def comenzar(self):
        self.timer.start()

    def sigue_andando(self):
        return self.timer.isActive()


class MiVentana(QWidget):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.timers = []
        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        # Definimos un montón de labels que corresponderán a un timer cada uno
        self.labels = {i : QLabel('Status: esperando timer', self) for i in range(1, 6)}
        self.boton = QPushButton('Ejecutar Timers', self)
        self.boton.clicked.connect(self.ejecutar_timers)

        hboxs = []
        for i in range(1, 6):
            hbox = QHBoxLayout()
            hbox.addStretch(1)
            hbox.addWidget(self.labels[i])
            hbox.addStretch(1)
            hboxs.append(hbox)

        hbox = QHBoxLayout()
        hbox.addStretch(1)
        hbox.addWidget(self.boton)
        hbox.addStretch(1)
        hboxs.append(hbox)

        vbox = QVBoxLayout()
        for hbox in hboxs:
            vbox.addStretch(1)
            vbox.addLayout(hbox)
        vbox.addStretch(1)
        self.setLayout(vbox)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle('Ejemplo timers')
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_timers(self):
        """
        Este método crea cinco threads cada vez que se presiona el botón en la
        interfaz. Los threads recibirás como argumento el índice del label 
        que les corresponde y el tiempo que toman entre cada iteración.
        """
        if len(self.timers) == 0 or not any([timer.sigue_andando() for timer in self.timers]):
            self.timers = []
            for i in range(1, 6):
                timer = MiTimers(i, i / 10)
                # Se conecta la señal emitida por el timer
                # a un método de la ventana
                timer.senal_actualizar.connect(self.actualizar_labels)
                self.timers.append(timer)
                timer.comenzar()

    def actualizar_labels(self, indice, texto):
        """
        Este método actualiza el label correspondiente según los datos 
        enviados desde un thread através del índice y aplica el texto.
        """
        self.labels[indice].setText(texto)


if __name__ == '__main__':
    app = QApplication([])
    form = MiVentana()
    sys.exit(app.exec_())

## `QThread`s y señales

En los ejemplos anteriores, se muestra el uso de `Thread`, `QThread` y `QTimer` que envían cambios a una ventana siempre mediante **señales**. Señales no es la única forma de conseguir este comportamiento, pero si es la **más escalable para generar cambios** de la interfaz gráfica. 

A continuación se muestra un ejemplo de `QThread` que modifica directamente la posición de labels dentro de una ventana. Se crean progresivamente múltiples *threads*, cada uno asociado a una etiqueta de comida distinta. La posición evoluciona al pasar el tiempo y en cada cambio el *thread* cambia **directamente** la posción de la etiqueta mediante `label.move()`. Si ejecutas en tu computador este código (recuerda copiar la carpeta de imagenes correspondiente) verás que funciona pero luego de unas centenas de *threads* creados, el programa **colapsa**.

In [ ]:
from PyQt5.QtWidgets import QLabel, QMainWindow, QApplication
from PyQt5.QtCore import QThread, QTimer
from PyQt5.QtGui import QPixmap
from time import sleep
from random import randint
from os import path


class Comida(QThread):

    def __init__(self, parent, limite_x, limite_y):
        """
        Una Comida es un QThread que movera una imagen de comida
        en una ventana. El __init__ recibe los parametros:
            parent: ventana
            limite_x e limite_y: Los límites rectangulares de la ventana
        """
        super().__init__()

        # Guardamos el path de la imagen que tendrá el Label
        self.ruta_imagen = path.join("img", "food", f"{randint(1, 9)}.png")
        
        # Creamos el Label y definimos su tamaño
        self.label = QLabel(parent)
        self.label.setGeometry(-50, -50, 50, 50)
        self.label.setPixmap(QPixmap(self.ruta_imagen))
        self.label.setScaledContents(True)
        self.label.setVisible(True)

        #Guardamos los limites de la ventana para que no pueda salirse de ella
        self.limite_x = limite_x
        self.limite_y = limite_y
        # Seteamos la posición inicial y la guardamos para usarla como una property
        self.__posicion = (0, 0)
        self.posicion = (randint(0, limite_x), randint(0, limite_y))

        self.label.show()
        self.start()

    @property
    def posicion(self):
        return self.__posicion

    # Cada vez que se actualicé la posición,
    # se actualiza la posición de la etiqueta
    @posicion.setter
    def posicion(self, valor):
        self.__posicion = valor
        self.label.move(*self.posicion)

    def run(self):
        while self.posicion[0] < self.limite_x \
            and self.posicion[1] < self.limite_y:
            sleep(0.1)
            nuevo_x = self.posicion[0] + 1
            nuevo_y = self.posicion[1] + 1
            self.posicion = (nuevo_x, nuevo_y)


class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setGeometry(200, 200, 500, 500)
        self.show()

        # Contador de cuanta comida hemos creado
        self.comida_creada = 0

        # Creamos un Timer que se encargara de crear la comida
        self.timer_crea_comida = QTimer(self)
        self.timer_crea_comida.setInterval(50)
        self.timer_crea_comida.timeout.connect(self.creador_de_comida)
        self.timer_crea_comida.start()

        self.comida = []

    def creador_de_comida(self):
        nueva_comida = Comida(self, self.width(), self.height())
        self.comida.append(nueva_comida)
        self.comida_creada += 1
        print(f"Has creado {self.comida_creada} unidades de comida\n")

if __name__ == '__main__':
    app = QApplication([])
    ex = MyWindow()
    app.exec_()

Esto se produce debido a que hay multiples *threads* haciendo casi simultaneamente cambios directos en la interfaz, que permite espacio para potenciales errores de concurrencia, que son clásicos en *threads*. 

En cambio, al usar señales se evade de mejor forma este problema ya que se delega al manejo de eventos de PyQt a que realize los cambios en la interfaz. El siguiente código muestra una adapatión del código anterior pero utilizando señales. Al ejecutarlo verás que se actualiza de mejor forma las etiquetas en la ventana y no se cae hasta los múltiples miles de *threads* iniciados:

In [ ]:
from PyQt5.QtWidgets import QLabel, QMainWindow, QApplication
from PyQt5.QtCore import QThread, QTimer, pyqtSignal
from PyQt5.QtGui import QPixmap
from time import sleep
from random import randint
from os import path


class Comida(QThread):

    actualizar = pyqtSignal(QLabel, int, int)

    def __init__(self, parent, limite_x, limite_y):
        """
        Una Comida es un QThread que movera una imagen de comida
        en una ventana. El __init__ recibe los parametros:
            parent: ventana
            limite_x e limite_y: Los límites rectangulares de la ventana
        """
        super().__init__()

        # Guardamos el path de la imagen que tendrá el Label
        self.ruta_imagen = path.join("img", "food", f"{randint(1, 9)}.png")
        
        # Creamos el Label y definimos su tamaño
        self.label = QLabel(parent)
        self.label.setGeometry(-50, -50, 50, 50)
        self.label.setPixmap(QPixmap(self.ruta_imagen))
        self.label.setScaledContents(True)
        self.label.setVisible(True)

        #Guardamos los limites de la ventana para que no pueda salirse de ella
        self.limite_x = limite_x
        self.limite_y = limite_y
        # Seteamos la posición inicial y la guardamos para usarla como una property
        self.__posicion = (0, 0)
        self.posicion = (randint(0, limite_x), randint(0, limite_y))

        self.label.show()
        self.start()

    @property
    def posicion(self):
        return self.__posicion

    # Cada vez que se actualicé la posición,
    # se actualiza la posición de la etiqueta
    @posicion.setter
    def posicion(self, valor):
        self.__posicion = valor
        self.actualizar.emit(self.label, *self.posicion)

    def run(self):
        while self.posicion[0] < self.limite_x \
            and self.posicion[1] < self.limite_y:
            sleep(0.1)
            nuevo_x = self.posicion[0] + 1
            nuevo_y = self.posicion[1] + 1
            self.posicion = (nuevo_x, nuevo_y)


class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setGeometry(200, 200, 500, 500)
        self.show()

        # Contador de cuanta comida hemos creado
        self.comida_creada = 0

        # Creamos un Timer que se encargara de crear la comida
        self.timer_crea_comida = QTimer(self)
        self.timer_crea_comida.setInterval(50)
        self.timer_crea_comida.timeout.connect(self.creador_de_comida)
        self.timer_crea_comida.start()

        self.comida = []

    def creador_de_comida(self):
        nueva_comida = Comida(self, self.width(), self.height())
        nueva_comida.actualizar.connect(self.actualizar_label)
        self.comida.append(nueva_comida)
        self.comida_creada += 1
        print(f"Has creado {self.comida_creada} unidades de comida\n")
    
    def actualizar_label(self, label, x, y):
        label.move(x, y)

if __name__ == '__main__':
    app = QApplication([])
    ex = MyWindow()
    app.exec_()


La conclusión de este experimento es que, en general, el uso de señales en conjunto a *theading* permite evadir potenciales problemas de concurrencia.